In [1]:
%pip install coremltools
%pip install timm
%pip install opencv-python
%pip install thop
%pip install pandas
%pip install seaborn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import torch

config = {
    'classical_sr' : {
        'args': {
            'upscale' : 2, # flexible
            'in_chans' : 3,
            'img_size' : 48,
            'window_size' : 8,
            'img_range' : 1.,
            'depths' :[6, 6, 6, 6, 6, 6],
            'embed_dim' : 180,
            'num_heads' : [6, 6, 6, 6, 6, 6],
            'mlp_ratio' : 2,
            'upsampler' : 'pixelshuffle',
            'resi_connection' : '1conv',
        },
        'path' : 'model_zoo/swinir/001_classicalSR_DIV2K_s48w8_SwinIR-M_x2.pth', #match with upscale factor
    },
    'device' : 'cuda' if torch.cuda.is_available() else 'cpu',

    'lightweight_sr' : {
        'args': {
            'upscale' : 2, # flexible
            'in_chans' : 3,
            'img_size' : 64,
            'window_size' : 8,
            'img_range' : 1.,
            'depths' :[6, 6, 6, 6],
            'embed_dim' : 60,
            'num_heads' : [6, 6, 6, 6],
            'mlp_ratio' : 2,
            'upsampler' : 'pixelshuffledirect',
            'resi_connection' : '1conv',
        },
        'path' : 'model_zoo/swinir/002_lightweightSR_DIV2K_s64w8_SwinIR-S_x2.pth', #match with upscale factor
    }
}

In [ ]:
from models.network_swinir import SwinIR as net

device = config['device']
TASK_NAME = 'lightweight_sr'


model_info = config[TASK_NAME]
model = net(**model_info['args'])
param_key_g = 'params'

pretrained_model = torch.load(model_info['path'])
model.load_state_dict(pretrained_model[param_key_g] if param_key_g in pretrained_model.keys() else pretrained_model, strict=True)
model.to(device)

SwinIR(
  (conv_first): Conv2d(3, 60, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (patch_embed): PatchEmbed(
    (norm): LayerNorm((60,), eps=1e-05, elementwise_affine=True)
  )
  (patch_unembed): PatchUnEmbed()
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0): RSTB(
      (residual_group): BasicLayer(
        dim=60, input_resolution=(64, 64), depth=6
        (blocks): ModuleList(
          (0): SwinTransformerBlock(
            dim=60, input_resolution=(64, 64), num_heads=6, window_size=8, shift_size=0, mlp_ratio=2
            (norm1): LayerNorm((60,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              dim=60, window_size=(8, 8), num_heads=6
              (qkv): Linear(in_features=60, out_features=180, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=60, out_features=60, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (s

In [94]:
import importlib
import pruning_utils as pu
importlib.reload(pu)


<module 'pruning_utils' from '/Users/vedantbhasin/Library/CloudStorage/GoogleDrive-vbhasin999@gmail.com/My Drive/SwinIR-experiments/pruning_utils.py'>

In [96]:
from torch.nn.utils.prune import global_unstructured, L1Unstructured
import copy
mod2prune = copy.deepcopy(model)

parameters_to_prune = [
    (module, name) for module in mod2prune.modules()
    for name, param in module.named_parameters(recurse=False)
]

orig_params = copy.deepcopy(parameters_to_prune)
n_iter = 0
for itr in range(n_iter):
    global_unstructured(
        parameters_to_prune,
        pruning_method=L1Unstructured,
        amount=0.33
    )

print(f'\noverall sparsity: {pu.calculate_sparsity_overall(mod2prune, parameters_to_prune):.4f}')
print(f'model size: {pu.calculate_sparse_model_size(mod2prune, temp_file=f"pruned_models/unstructured_itr_{n_iter}"):.2f} MB')

# pruning_data_unstructured['Iteration'].append(itr + 1)
# pruning_data_unstructured['Sparsity'].append(pu.calculate_sparsity_overall(mod2prune, parameters_to_prune))
# pruning_data_unstructured['Disk Size (MB)'].append(pu.calculate_sparse_model_size(mod2prune))




overall sparsity: 0.0000
model sparsity: 0.0000
model size: 34.07 MB


In [85]:
print(f'model size: {pu.calculate_sparse_model_size(model):.2f} MB')

model sparsity: 0.0000
model size: 34.07 MB


In [ ]:
pruning_data_unstructured = {
    "Iteration" : [0, 1, 2, 3, 4, 5],
    "Sparsity" : [0.0, 0.33, 0.5511, 0.6992, 0.7985, 0.8650],
    "Accuracy" : [],
    "Latency (s)" : [],
    "Disk Size (MB)" : [34.07, 27.23, 22.52, 19.56, 17.65, 16.40],
}